# Peak Hour Detection

In [59]:
import math
import pandas as pd
import numpy as np
import datetime

In [223]:
file_up_fs = 'G:\\Modeling Endogeneity\\Data\\200m_2lanes_up_fs_1.xlsx'
data = pd.read_excel(file_up_fs)

In [224]:
data = data.iloc[:,0:7]
data.columns = ['t','f1','s1','f2','s2','f','s']
data = data[['t','s']]
data.describe()

,s
count,2016.000000
mean,64.233135
std,1.479378
min,57.100000
25%,63.700000
50%,64.400000
75%,64.800000
max,69.400000


In [225]:
data['t'] = pd.to_datetime(data['t'])
date_rng = pd.date_range(start='1/8/2017', end='1/15/2017', freq='H', closed='left')
speed = pd.DataFrame(date_rng, columns=['t'])
speed['s'] = np.random.ranf(168)
for day in range(7):
    begin = data['t'][0] + datetime.timedelta(days=day)
    end = data['t'][0] + datetime.timedelta(days=day+1)
    interval = (data['t']>begin) & (data['t']<end)
    data_day = data[interval]
    for hour in range(24):
        data_hour = data_day[data_day['t'].dt.hour == hour]
        s_mean = np.mean(data_hour)
        speed['s'][(24*day)+hour] = s_mean

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [226]:
k = 4
h = 1.5
a = speed[(speed['t'].dt.hour>3) & (speed['t'].dt.hour<20)]
# a = a.reset_index(drop=True)
# a['s'] = np.random.ranf(112)
for day in range(7):
    for hour in range(16):
        i = 24*day+hour+4
        s1 = max(speed['s'][i]-speed['s'][i-1], speed['s'][i]-speed['s'][i-2],\
                     speed['s'][i]-speed['s'][i-3], speed['s'][i]-speed['s'][i-4])
        s2 = max(speed['s'][i]-speed['s'][i+1], speed['s'][i]-speed['s'][i+2],\
                     speed['s'][i]-speed['s'][i+3], speed['s'][i]-speed['s'][i+4])
        s = (s1+s2)/2
        a['s'][i] = s

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [227]:
a = a.reset_index(drop=True)

m = pd.DataFrame(np.random.ranf(14).reshape((7,2)),columns=['mean','std'])
for day in range(7):    
    m.loc[day,'mean'] = a.loc[day*16:((day+1)*16),'s'].mean()
    m.loc[day,'std'] = a.loc[day*16:((day+1)*16),'s'].std()

In [228]:
a['t'][25]

Timestamp('2017-01-09 13:00:00')

In [229]:
peak_set = pd.DataFrame(columns=['peakhour'])
for day in range(7):
    mean = m['mean'][day]
    std = m['std'][day]
    for hour in range(16):
        i = 16*day+hour
        if (a['s'][i]>0) & ((a['s'][i]-mean)>(h*std)):
            peak_set.loc[i] = a['t'][i]

In [230]:
peak_set

,peakhour
25,2017-01-09 13:00:00
26,2017-01-09 14:00:00
35,2017-01-10 07:00:00
36,2017-01-10 08:00:00
51,2017-01-11 07:00:00
66,2017-01-12 06:00:00
105,2017-01-14 13:00:00


In [222]:
peak_set

,peakhour
0,2017-01-08 04:00:00
30,2017-01-09 18:00:00
47,2017-01-10 19:00:00
62,2017-01-11 18:00:00
79,2017-01-12 19:00:00
83,2017-01-13 07:00:00
105,2017-01-14 13:00:00
